In [26]:
import json
import os
import pandas as pd


def load_json(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

data_path = '/Users/ahmed/Desktop/msc-24/ECHR/echr-processed/'
df1 = pd.read_csv('/Users/ahmed/Desktop/msc-24/TND/kc_classification_data/pre_cutoff_data/df_1.csv')
df2 = pd.read_csv('/Users/ahmed/Desktop/msc-24/TND/kc_classification_data/pre_cutoff_data/df_2.csv')
df3 = pd.read_csv('/Users/ahmed/Desktop/msc-24/TND/kc_classification_data/pre_cutoff_data/df_3.csv')
df4 = pd.read_csv('/Users/ahmed/Desktop/msc-24/TND/kc_classification_data/pre_cutoff_data/df_4.csv')


data = {}
app_to_item = {}
item_to_app = {}
cases_path = os.listdir(data_path)


#combine all data frames
df = pd.concat([df1, df2, df3, df4])
file_paths = df['file_path'].tolist()

for case_path in cases_path:
    case = load_json(data_path + case_path)
    appno  = case['appno']
    title = case['docname']
    extracted_apps = case['extractedappno']
    importance = case['importance']
    article = case['article']
    itemid = case_path.split('.')[0]

    if ';' in appno:
        appno_list = appno.split(';')
        for app in appno_list:
            app_to_item[app] = itemid
    else:
        app_to_item[appno] = itemid
    item_to_app[itemid] = appno
    data[itemid] = {'case_path':case_path, 'title':title, 'extracted_apps':extracted_apps, 'importance':importance, 'article':article}

In [27]:
app_to_item

{'27243/95': '001-60310',
 '21890/03': '001-82179',
 '6544/05': '001-93292',
 '24139/20': '001-217658',
 '30754/04': '001-93768',
 '47152/06': '001-161822',
 '71572/01': '001-101223',
 '25771/03': '001-79131',
 '74507/01': '001-82483',
 '49158/99': '001-61852',
 '37685/02': '001-80514',
 '46284/99': '001-67085',
 '44765/08': '001-210037',
 '10607/10': '001-210037',
 '38903/16': '001-222146',
 '39614/16': '001-222146',
 '39741/16': '001-222146',
 '40693/16': '001-222146',
 '40863/16': '001-222146',
 '43443/16': '001-222146',
 '50279/16': '001-222146',
 '60932/16': '001-222146',
 '61290/16': '001-222146',
 '63340/16': '001-222146',
 '56793/00': '001-76694',
 '50064/18': '001-224991',
 '54135/18': '001-224991',
 '6674/19': '001-224991',
 '6719/19': '001-224991',
 '6730/19': '001-224991',
 '6949/19': '001-224991',
 '6975/19': '001-224991',
 '7043/19': '001-224991',
 '7335/19': '001-224991',
 '7391/19': '001-224991',
 '7394/19': '001-224991',
 '7419/19': '001-224991',
 '7421/19': '001-22499

In [73]:
# load a case
file_path = '/Users/ahmed/Desktop/msc-24/ECHR/echr-processed/'
case = load_json(file_path + '001-220960.json')

In [74]:
cited_cases_appno = case['extractedappno']

In [75]:
for cca in cited_cases_appno:
    if cca in app_to_item:
        print(cca, app_to_item[cca])
    else:
        print(cca, ' ------- not in app_to_item')

5084/18 001-220960
8927/11 001-165250
41216/13 001-191703
66069/09 001-122664
57592/08 001-170347
498/10  ------- not in app_to_item
21906/04 001-85019
42750/09 001-112108
43522/98  ------- not in app_to_item
22318/02  ------- not in app_to_item
28578/03  ------- not in app_to_item
48058/09  ------- not in app_to_item
38621/97  ------- not in app_to_item
2440/07 001-89161
25978/07 001-97425
31240/03 001-103663
34979/97  ------- not in app_to_item
30500/11 001-173776
72508/13 001-178753


In [39]:
import re

def split_into_paragraphs(text: str) -> list[str]:
    """
    Split ECHR case text into paragraphs based on numbered starts.
    Handles both numbered paragraphs (e.g., "2.") and paragraphs starting with just a period.
    """
    # Remove extra whitespace and normalize newlines
    text = text.strip()
    
    # Split pattern: Look for either:
    # 1. A number followed by a period (\d+\.)
    # 2. Just a period at the start of a line (^\.)
    # Use positive lookbehind to keep the delimiter in the result
    paragraphs = re.split(r'(?<=\n)(?=\d+\.|\.|$)', text)
    
    # Clean up paragraphs
    cleaned_paragraphs = []
    for para in paragraphs:
        # Remove empty paragraphs and clean whitespace
        para = para.strip()
        if para:
            cleaned_paragraphs.append(para)
    
    return cleaned_paragraphs

In [40]:
text = case['facts'] + case['other']+ case['law']

paragraphs = split_into_paragraphs(text)

In [47]:
app_patterns = [re.escape(app_no) for app_no in cited_cases_appno]
app_pattern = re.compile('|'.join(app_patterns))

In [50]:
matching_paragraphs = []

for para in paragraphs:
    found_app_nos = app_pattern.findall(para)
    if found_app_nos:
        matching_paragraphs.append((para, found_app_nos))

In [51]:
matching_paragraphs

[('27. By a decision of 29 December 1999, the Constitutional Court of Ukraine abolished the death penalty and the parliament passed Law no. 1483 of 22 February 2000 amending the Criminal Code; the death penalty, as a punishment for some crimes, was replaced with life imprisonment. The amendments entered into force on 29 March 2000 (see, for more details, Ruban v. Ukraine, no. 8927/11, 20-22, 12 July 2016).\n2001 Criminal Code',
  ['8927/11']),
 ('32. In 2019 the Court found a violation of Article 3 of the Convention in Petukhov v. Ukraine (no. 2) (no. 41216/13, 12 March 2019) in respect of insufficient medical assistance provided to Mr Petukhov and the irreducibility of his life sentence. Mr Petukhov sought review of his case before the Supreme Court on the basis of the Court s judgment. The Grand Chamber of the Supreme Court rejected his request. In particular, with regard to the irreducibility of his life sentence, the Grand Chamber of the Supreme Court referred to the Court s conclu

In [36]:
import re
from dataclasses import dataclass
from typing import List, Dict, Optional

@dataclass
class Citation:
    case_name: str
    app_no: Optional[str]
    paragraphs: str
    date: Optional[str]
    is_full_citation: bool

class ECHRCitationExtractor:
    def __init__(self, known_app_numbers: List[str]):
        self.known_app_numbers = known_app_numbers
        self.case_mapping: Dict[str, str] = {}  # Maps case names to app numbers
        
    def extract_citations(self, text: str) -> List[Citation]:
        citations = []
        
        # Pattern for full citations with various formats
        full_patterns = [
            # Standard format with optional prefixes and (dec.)
            r'(?:see\s+)?([A-Za-zíñ\s]+(?:\s+\(no\.\s*\d+\))?\s+(?:v\.\s+[A-Za-z\s]+)?)\s*(?:\[GC\])?,\s*(?:nos\.\s*(\d+/\d+(?:\s+and\s+\d+\s+others)?)|no\.\s*(\d+/\d+)),\s*([0-9\-,\s]+)(?:,\s*(?:ECHR\s+\d{4}(?:\s+\(extracts\))?|(\d+\s+[A-Za-z]+\s+\d{4})))?',
            
            # Format with (dec.) in the middle
            r'(?:see\s+)?([A-Za-zíñ\s]+)\s*\(dec\.\),\s*no\.\s*(\d+/\d+),\s*([0-9\-,\s]+)(?:,\s*(?:ECHR\s+\d{4}(?:\s+\(extracts\))?|(\d+\s+[A-Za-z]+\s+\d{4})))?',
            
            # Format with multiple application numbers
            r'(?:see\s+)?([A-Za-zíñ\s]+)\s*\[GC\],\s*nos\.\s*(\d+/\d+(?:\s+and\s+\d+\s+others)?),\s*([0-9\-,\s]+)(?:,\s*(?:ECHR\s+\d{4}(?:\s+\(extracts\))?|(\d+\s+[A-Za-z]+\s+\d{4})))?'
        ]
        
        # Pattern for subsequent citations
        subsequent_pattern = r'([A-Za-zíñ\s]+),\s*(?:cited above,\s*([0-9\-,\s]+)|([0-9\-,\s]+),\s*cited above)'
        
        # Find all full citations using multiple patterns
        for pattern in full_patterns:
            for match in re.finditer(pattern, text):
                case_name = match.group(1).strip()
                # Handle multiple possible groups for app_no
                app_no = next((g for g in match.groups()[1:] if g and any(known in g for known in self.known_app_numbers)), None)
                
                if app_no:
                    # Extract just the first application number if multiple are present
                    app_no = re.search(r'\d+/\d+', app_no).group(0)
                    
                # Find the paragraphs and date in the remaining groups
                paragraphs = next((g for g in match.groups() if g and re.match(r'^[0-9\-,\s]+$', g)), '')
                date = next((g for g in match.groups() if g and re.match(r'\d+\s+[A-Za-z]+\s+\d{4}', str(g))), None)
                
                # Only add if we found a known application number
                if app_no in self.known_app_numbers:
                    self.case_mapping[self._normalize_case_name(case_name)] = app_no
                    citations.append(Citation(
                        case_name=case_name,
                        app_no=app_no,
                        paragraphs=paragraphs,
                        date=date,
                        is_full_citation=True
                    ))
        
        # Find all subsequent citations
        for match in re.finditer(subsequent_pattern, text):
            case_name = match.group(1).strip()
            paragraphs = match.group(2) if match.group(2) else match.group(3)
            
            normalized_name = self._normalize_case_name(case_name)
            if normalized_name in self.case_mapping:
                citations.append(Citation(
                    case_name=case_name,
                    app_no=self.case_mapping[normalized_name],
                    paragraphs=paragraphs,
                    date=None,
                    is_full_citation=False
                ))
        
        return citations
    
    def _normalize_case_name(self, case_name: str) -> str:
        """Normalize case name for comparison by removing common variations."""
        # Remove "(no. 2)" and similar suffixes
        case_name = re.sub(r'\s*\(no\.\s*\d+\)', '', case_name)
        # Remove "(dec.)" suffix
        case_name = re.sub(r'\s*\(dec\.\)', '', case_name)
        return re.sub(r'\s+', ' ', case_name.lower().strip())

In [37]:
# Example usage
    # List of known application numbers
known_app_numbers =case['extractedappno']

# Sample text
text = case['facts'] + case['other']+ case['law']

extractor = ECHRCitationExtractor(known_app_numbers)
citations = extractor.extract_citations(text)

# Print results
for citation in citations:
    if citation.is_full_citation:
        print(f"Full citation: {citation.case_name} (no. {citation.app_no}), §§ {citation.paragraphs}")
    else:
        print(f"Subsequent citation: {citation.case_name} (no. {citation.app_no}), §§ {citation.paragraphs}")


Full citation: Ruban v. Ukraine (no. 8927/11), §§ 20-22, 12 
Full citation: he referred to Vinter and Others v. the United Kingdom (no. 66069/09), §§ 124-30, 
Full citation: Hutchinson v. the United Kingdom (no. 57592/08), §§ 42-45, 17 
Full citation: Kafkaris v. Cyprus (no. 21906/04), §§ 142, 
Full citation: and Del R o Prada v. Spain (no. 42750/09), §§ 83, 
Full citation: Grava v. Italy (no. 43522/98), §§ 51, 10 
Full citation: Soldatenko v. Ukraine (no. 2440/07), §§ 23 
Full citation: Garkavyy v. Ukraine (no. 25978/07), §§ 18 
Full citation: Zhukovskiy v. Ukraine (no. 31240/03), §§ 3 
Full citation: Malik Babayev v. Azerbaijan (no. 30500/11), §§ 97, 1 
Full citation: Merabishvili v. Georgia (no. 72508/13), §§ 372, 28 
Full citation: Poland (no. 498/10), §§ 32-33, 23 
Full citation: Csosz nszki v Sweden (no. 22318/02), §§ 27 
Full citation: Sweden (no. 28578/03), §§ 27 
Full citation: the Czech Republic (no. 48058/09), §§ 6 
Full citation: the Czech Republic (no. 38621/97), §§  
Full

In [38]:
all_apps = ' '.join(known_app_numbers)
for citation in citations:
    if citation.is_full_citation:
        if citation.app_no in all_apps:
            print(f"Full citation: {citation.case_name} (no. {citation.app_no}), §§ {citation.paragraphs}")
        else:
            print(f"Full citation: {citation.case_name} (no. {citation.app_no}), §§ {citation.paragraphs} ---- not in known apps")

Full citation: Ruban v. Ukraine (no. 8927/11), §§ 20-22, 12 
Full citation: he referred to Vinter and Others v. the United Kingdom (no. 66069/09), §§ 124-30, 
Full citation: Hutchinson v. the United Kingdom (no. 57592/08), §§ 42-45, 17 
Full citation: Kafkaris v. Cyprus (no. 21906/04), §§ 142, 
Full citation: and Del R o Prada v. Spain (no. 42750/09), §§ 83, 
Full citation: Grava v. Italy (no. 43522/98), §§ 51, 10 
Full citation: Soldatenko v. Ukraine (no. 2440/07), §§ 23 
Full citation: Garkavyy v. Ukraine (no. 25978/07), §§ 18 
Full citation: Zhukovskiy v. Ukraine (no. 31240/03), §§ 3 
Full citation: Malik Babayev v. Azerbaijan (no. 30500/11), §§ 97, 1 
Full citation: Merabishvili v. Georgia (no. 72508/13), §§ 372, 28 
Full citation: Poland (no. 498/10), §§ 32-33, 23 
Full citation: Csosz nszki v Sweden (no. 22318/02), §§ 27 
Full citation: Sweden (no. 28578/03), §§ 27 
Full citation: the Czech Republic (no. 48058/09), §§ 6 
Full citation: the Czech Republic (no. 38621/97), §§  
Full

In [35]:
['8927/11', '66069/09', '57592/08', '21906/04']

['5084/18',
 '8927/11',
 '41216/13',
 '66069/09',
 '57592/08',
 '498/10',
 '21906/04',
 '42750/09',
 '43522/98',
 '22318/02',
 '28578/03',
 '48058/09',
 '38621/97',
 '2440/07',
 '25978/07',
 '31240/03',
 '34979/97',
 '30500/11',
 '72508/13']

In [60]:
ddf = pd.read_csv('/Users/ahmed/Downloads/echr_case_paragraphs.csv')

In [65]:
ddf.shape

(1024702, 4)

In [63]:
ddf.head(3)

,case_id,case_name,paragraph_number,paragraph_text
0,001-95845,CASE OF POPOVIC v. SERBIA,1,1. The case originated in an application (no....
1,001-95845,CASE OF POPOVIC v. SERBIA,2,"2. As of 3 June 2006, following the Montenegr..."
2,001-95845,CASE OF POPOVIC v. SERBIA,3,3. The applicant was represented by Mr S. Sto...


In [72]:
len(list(set(ddf['case_id'].tolist()) - set(list(item_to_app.keys()))))

162

In [77]:
(list(set(ddf['case_id'].tolist()) - set(list(item_to_app.keys()))))

['001-68715',
 '001-154265',
 '001-225329',
 '001-193746',
 '001-58015',
 '001-191117',
 '001-214296',
 '001-86490',
 '001-213713',
 '001-225209',
 '001-189902',
 '001-127056',
 '001-221503',
 '001-188649',
 '001-215724',
 '001-80046',
 '001-226385',
 '001-115714',
 '001-184888',
 '001-222311',
 '001-213058',
 '001-223023',
 '001-194885',
 '001-223716',
 '001-159017',
 '001-178106',
 '001-200886',
 '001-57920',
 '001-97689',
 '001-218516',
 '001-209039',
 '001-57625',
 '001-226442',
 '001-141293',
 '001-142672',
 '001-116416',
 '001-122885',
 '001-177070',
 '001-178082',
 '001-57671',
 '001-45780',
 '001-212689',
 '001-220664',
 '001-228854',
 '001-57660',
 '001-160248',
 '001-219563',
 '001-22159',
 '001-219984',
 '001-217535',
 '001-220073',
 '001-198384',
 '001-100411',
 '001-216400',
 '001-213213',
 '001-203885',
 '001-57982',
 '001-223709',
 '001-160441',
 '001-211178',
 '001-224559',
 '001-224959',
 '001-189777',
 '001-211972',
 '001-58140',
 '001-216018',
 '001-105779',
 '001-20

In [78]:
app_to_item['54825/00']

KeyError: '54825/00'